In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ResNet Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(64, 64, 2, stride=1)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.fc = nn.Linear(1152, 10)

    def _make_layer(self, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        for _ in range(num_blocks - 1):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = nn.functional.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [20]:
train_data, train_labels = torch.load('dataset/mnist/training.pt')
test_data, test_labels = torch.load('dataset/mnist/test.pt')

train_dataset = TensorDataset(train_data.unsqueeze(1).float(), train_labels)
test_dataset = TensorDataset(test_data.unsqueeze(1).float(), test_labels)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


In [21]:
# Initialize model, loss, and optimizer
model = ResNet().to(device)  # Move model to GPU
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [23]:
# Training
for epoch in range(10):
    model.train()
    for i, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)  # Move data and labels to GPU
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/10], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

# Testing
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, labels in test_loader:
        data, labels = data.to(device), labels.to(device)  # Move data and labels to GPU
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")



Epoch [1/10], Step [100/469], Loss: 0.0565
Epoch [1/10], Step [200/469], Loss: 0.0999
Epoch [1/10], Step [300/469], Loss: 0.0161
Epoch [1/10], Step [400/469], Loss: 0.0347
Epoch [2/10], Step [100/469], Loss: 0.0115
Epoch [2/10], Step [200/469], Loss: 0.0085
Epoch [2/10], Step [300/469], Loss: 0.0426
Epoch [2/10], Step [400/469], Loss: 0.0088
Epoch [3/10], Step [100/469], Loss: 0.0122
Epoch [3/10], Step [200/469], Loss: 0.0083
Epoch [3/10], Step [300/469], Loss: 0.1165
Epoch [3/10], Step [400/469], Loss: 0.0187
Epoch [4/10], Step [100/469], Loss: 0.0080
Epoch [4/10], Step [200/469], Loss: 0.0223
Epoch [4/10], Step [300/469], Loss: 0.0511
Epoch [4/10], Step [400/469], Loss: 0.0215
Epoch [5/10], Step [100/469], Loss: 0.0045
Epoch [5/10], Step [200/469], Loss: 0.0228
Epoch [5/10], Step [300/469], Loss: 0.0036
Epoch [5/10], Step [400/469], Loss: 0.0376
Epoch [6/10], Step [100/469], Loss: 0.0101
Epoch [6/10], Step [200/469], Loss: 0.0290
Epoch [6/10], Step [300/469], Loss: 0.0190
Epoch [6/10

In [17]:
device

device(type='cuda')

In [24]:
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNo